In [1]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image


In [2]:
batchsize=8

In [3]:

train_datagen= ImageDataGenerator(rescale= 1./255, rotation_range=0.2, shear_range=0.2, zoom_range=0.2, width_shift_range=0.2,
                                 height_shift_range=0.2, validation_split=0.2)

In [4]:

train_data = train_datagen.flow_from_directory(r'C:\Users\INFINIX\Driver Drowsiness Detection System\Prepared_Data\train', target_size=(80,80),
                                               batch_size=batchsize, class_mode='categorical', subset='training')

validation_data = train_datagen.flow_from_directory(r'C:\Users\INFINIX\Driver Drowsiness Detection System\Prepared_Data\train', target_size=(80,80),
                                               batch_size=batchsize, class_mode='categorical', subset='validation')

Found 64105 images belonging to 2 classes.
Found 16025 images belonging to 2 classes.


In [5]:
test_datagen= ImageDataGenerator(rescale= 1./255)

test_data = test_datagen.flow_from_directory(r'C:\Users\INFINIX\Driver Drowsiness Detection System\Prepared_Data\test', target_size=(80,80),
                                               batch_size=batchsize, class_mode='categorical')

Found 4768 images belonging to 2 classes.


In [6]:
bmodel= InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel= bmodel.output
hmodel= Flatten()(hmodel)
hmodel= Dense(64, activation='relu')(hmodel)
hmodel= Dropout(0.5)(hmodel)
hmodel= Dense(2, activation='softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs=hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [7]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 80, 80, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 39, 39, 32)           864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 39, 39, 32)           96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 39, 39, 32)           0         ['batch_normalization[0][0

In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau


In [9]:
checkpoint = ModelCheckpoint(r'C:\Users\INFINIX\Driver Drowsiness Detection System\models\model.h5',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]

In [10]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=5)

C:\Users\INFINIX\AppData\Local\Temp\ipykernel_12652\3918530525.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,


Epoch 1/5


8013/8013 [==============================] - ETA: 0s - loss: 0.2099 - accuracy: 0.9176
Epoch 1: val_loss improved from inf to 0.20229, saving model to C:\Users\INFINIX\Driver Drowsiness Detection System\models\model.h5


C:\Users\INFINIX\.conda\envs\yogesh\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8013/8013 [==============================] - 1266s 157ms/step - loss: 0.2099 - accuracy: 0.9176 - val_loss: 0.2023 - val_accuracy: 0.9109 - lr: 0.0010
Epoch 2/5
8013/8013 [==============================] - ETA: 0s - loss: 0.1777 - accuracy: 0.9318
Epoch 2: val_loss improved from 0.20229 to 0.17742, saving model to C:\Users\INFINIX\Driver Drowsiness Detection System\models\model.h5
8013/8013 [==============================] - 1182s 148ms/step - loss: 0.1777 - accuracy: 0.9318 - val_loss: 0.1774 - val_accuracy: 0.9347 - lr: 0.0010
Epoch 3/5
8013/8013 [==============================] - ETA: 0s - loss: 0.1709 - accuracy: 0.9351
Epoch 3: val_loss did not improve from 0.17742
8013/8013 [==============================] - 997s 124ms/step - loss: 0.1709 - accuracy: 0.9351 - val_loss: 0.1900 - val_accuracy: 0.9212 - lr: 0.0010
Epoch 4/5
8013/8013 [==============================] - ETA: 0s - loss: 0.1656 - accuracy: 0.9370
Epoch 4: val_loss did not improve from 0.17742
8013/8013 [================

In [11]:
#model Evaluation

In [12]:
acc_tr, loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

C:\Users\INFINIX\AppData\Local\Temp\ipykernel_12652\713691994.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_tr, loss_tr = model.evaluate_generator(train_data)


0.14006812870502472
0.9469308257102966


In [13]:
acc_vr, loss_vr = model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)

C:\Users\INFINIX\AppData\Local\Temp\ipykernel_12652\4081756742.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_vr, loss_vr = model.evaluate_generator(validation_data)


0.1913442462682724
0.9203743934631348


In [14]:
acc_test, loss_test= model.evaluate_generator(test_data)
print(acc_test)
print(loss_test)

C:\Users\INFINIX\AppData\Local\Temp\ipykernel_12652\3161718765.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_test, loss_test= model.evaluate_generator(test_data)


0.43353593349456787
0.8496224880218506
